# Import block

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D # for 3d plotting
import os

import h5py
import itertools as it
import datetime

%matplotlib inline

Using TensorFlow backend.


# Load data

In [2]:
# load the data
with h5py.File(os.getcwd() + '\\input\\full_dataset_vectors.h5', 'r') as hf:
    x_train_raw = hf["X_train"][:]
    y_train_raw = hf["y_train"][:]
    x_test_raw = hf["X_test"][:]
    y_test_raw = hf["y_test"][:]


# length check
assert(len(x_train_raw) == len(y_train_raw))
assert(len(x_test_raw) == len(y_test_raw))

print ("x_train_raw shape: ", x_train_raw.shape)
print ("y_train_raw shape: ", y_train_raw.shape)

print ("x_test_raw shape:  ", x_test_raw.shape)
print ("y_test_raw shape:  ", y_test_raw.shape)

x_train_raw shape:  (10000, 4096)
y_train_raw shape:  (10000,)
x_test_raw shape:   (2000, 4096)
y_test_raw shape:   (2000,)


# Convert to RGB values
Now, lets implement a 3D convolutional Neural network on this dataset. To use 2D convolutions, we first convert every image into a 3D shape : width, height, channels. Channels represents the slices of Red, Green, and Blue layers. So it is set as 3. In the similar manner, we will convert the input dataset into 4D shape in order to use 3D convolution for : length, breadth, height, channel (r/g/b)

In [3]:
# 1D vector to rgb values, provided by ../input/plot3d.py
def array_to_color(array, cmap="Oranges"):
    s_m = plt.cm.ScalarMappable(cmap=cmap)
    return s_m.to_rgba(array)[:,:-1]

# Transform data from 1d to 3d rgb
def rgb_data_transform(data):
    data_t = []
    for i in range(data.shape[0]):
        data_t.append(array_to_color(data[i]).reshape(16, 16, 16, 3))
    return np.asarray(data_t, dtype=np.float32)

In [4]:
n_classes = 10 # from 0 to 9, 10 labels totally

x_train = rgb_data_transform(x_train_raw)
x_test = rgb_data_transform(x_test_raw)

## convert target variable into one-hot
y_train = to_categorical(y_train_raw, n_classes)
y_test = to_categorical(y_test_raw, n_classes)

In [5]:
with tf.name_scope('inputs'):
    x_input = tf.placeholder(tf.float32, shape=[None, 16, 16, 16, 3])
    y_input = tf.placeholder(tf.float32, shape=[None, n_classes]) 

print ("x_train shape: ", x_train.shape)
print ("y_train shape: ", y_train.shape)

print ("x_test shape:  ", x_test.shape)
print ("y_test shape:  ", y_test.shape)

x_train shape:  (10000, 16, 16, 16, 3)
y_train shape:  (10000, 10)
x_test shape:   (2000, 16, 16, 16, 3)
y_test shape:   (2000, 10)


# Define CNN model

In [6]:
def cnn_model(x_train_data, keep_rate=0.5, seed=None, kernel_size_1=5, kernel_size_2=3, pool_size=3, 
              dense_activation1='relu', dense_activation2='relu'):
    
    with tf.name_scope("layer_a"):
        # conv => 16*16*16
        conv1 = tf.layers.conv3d(inputs=x_train_data, filters=16, kernel_size=[kernel_size_1,kernel_size_1,kernel_size_1], padding='same', activation=tf.nn.relu)
        # conv => 16*16*16
        conv2 = tf.layers.conv3d(inputs=conv1, filters=32, kernel_size=[kernel_size_2,kernel_size_2,kernel_size_2], padding='same', activation=tf.nn.relu)
        # pool => 8*8*8
        pool3 = tf.layers.max_pooling3d(inputs=conv2, pool_size=[pool_size, pool_size, pool_size], strides=pool_size)
        
    with tf.name_scope("layer_c"):
        # conv => 8*8*8
        conv4 = tf.layers.conv3d(inputs=pool3, filters=64, kernel_size=[kernel_size_1,kernel_size_1,kernel_size_1], padding='same', activation=tf.nn.relu)
        # conv => 8*8*8
        conv5 = tf.layers.conv3d(inputs=conv4, filters=128, kernel_size=[kernel_size_2,kernel_size_2,kernel_size_2], padding='same', activation=tf.nn.relu)
        # pool => 4*4*4
        pool6 = tf.layers.max_pooling3d(inputs=conv5, pool_size=[pool_size, pool_size, pool_size], strides=pool_size)
        
    with tf.name_scope("batch_norm"):
        cnn3d_bn = tf.layers.batch_normalization(inputs=pool6, training=True)
        
    with tf.name_scope("fully_con"):
        flattening = tf.reshape(cnn3d_bn, [-1, cnn3d_bn.shape[1]*cnn3d_bn.shape[2]*cnn3d_bn.shape[3]*cnn3d_bn.shape[4]])
        dense = tf.layers.dense(inputs=flattening, units=2048, activation=dense_activation1)
        dense = tf.layers.dense(inputs=flattening, units=512, activation=dense_activation2)
        # (1-keep_rate) is the probability that the node will be kept
        dropout = tf.layers.dropout(inputs=dense, rate=keep_rate, training=True)
        
    with tf.name_scope("y_conv"):
        y_conv = tf.layers.dense(inputs=dropout, units=10)
    
    return y_conv

# Define function to train and evaluate NN

In [7]:
def train_neural_network(x_train_data, y_train_data, x_test_data, y_test_data, **kwargs):
    
    learning_rate = kwargs.get('learning_rate',0.05)
    keep_rate = kwargs.get('keep_rate',0.7)
    epochs = kwargs.get('epochs',100)
    batch_size = kwargs.get('batch_size',128)
    kernel_size_1 = kwargs.get('kernel_size_1',5)
    kernel_size_2 = kwargs.get('kernel_size_2',3)
    pool_size = kwargs.get('pool_size',3)
    dense_activation1 = kwargs.get('dense_activation1','relu')
    dense_activation2 = kwargs.get('dense_activation2','relu')

    with tf.name_scope("cross_entropy"):
        prediction = cnn_model(x_input, keep_rate, 1, kernel_size_1, kernel_size_2, 
                               pool_size, dense_activation1, dense_activation2)
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y_input))
                              
    with tf.name_scope("training"):
        optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
           
    correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y_input, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
    
    iterations = int(len(x_train_data)/batch_size) + 1
    
    print(kwargs)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        import datetime

        start_time = datetime.datetime.now()

        iterations = int(len(x_train_data)/batch_size) + 1
        # run epochs
        for epoch in range(epochs):
            start_time_epoch = datetime.datetime.now()
            epoch_loss = 0
            # mini batch
            for itr in range(iterations):
                mini_batch_x = x_train_data[itr*batch_size: (itr+1)*batch_size]
                mini_batch_y = y_train_data[itr*batch_size: (itr+1)*batch_size]
                _optimizer, _cost = sess.run([optimizer, cost], feed_dict={x_input: mini_batch_x, y_input: mini_batch_y})
                epoch_loss += _cost

            #  using mini batch in case not enough memory
            acc = 0
            itrs = int(len(x_test_data)/batch_size) + 1
            for itr in range(itrs):
                mini_batch_x_test = x_test_data[itr*batch_size: (itr+1)*batch_size]
                mini_batch_y_test = y_test_data[itr*batch_size: (itr+1)*batch_size]
                acc += sess.run(accuracy, feed_dict={x_input: mini_batch_x_test, y_input: mini_batch_y_test})
            
            if(epoch % int(epochs/10) == 0):
                print(int((epoch/epochs)*100),'% complete')

        end_time = datetime.datetime.now()
        print('Testing Set Accuracy:',acc/itrs)
        print('Time elapsed: ', str(end_time - start_time))
        
    return acc/itrs

# Create a dict to allow all possible combos of hyperparameters

In [8]:
#####################
#####################
# STUDENT CODE HERE #
#####################
#####################

# Prepare the Grid
param_grid = dict(learning_rate=[0.0001], 
                  keep_rate=[0.2],
                  epochs=[201],
                  batch_size=[128],
                  kernel_size_1=[3], 
                  kernel_size_2=[3], 
                  pool_size=[2],
                  dense_activation1=['relu'],
                  dense_activation2=['sigmoid']
                  )

keys = param_grid.keys()
values = (param_grid[key] for key in keys)
combinations = [dict(zip(keys, combination)) for combination in it.product(*values)]

# Try all possible combs of hyperparameters and report best

In [9]:
accuracies = []
index = 0

start_time = datetime.datetime.now()

for combo in combinations:
    index = index + 1
    print()
    print('--',index,'-of-',len(combinations),'-----------------------------------------------------------------------------------')
    accuracies.append(train_neural_network(x_train[:100], y_train[:100], x_test[:100], y_test[:100], **combo))
    
end_time = datetime.datetime.now()    

# Overwrite index
index = np.argmax(accuracies)
print('Index -', index)
print('Value -', accuracies[index])
print('Hyper-parameters', combinations[index])
print('Total Run Time: ', str(end_time - start_time))

W0729 09:47:09.215701  6944 deprecation.py:323] From <ipython-input-6-948e11d448d3>:6: conv3d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv3D` instead.
W0729 09:47:09.218691  6944 deprecation.py:506] From C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor



-- 1 -of- 1 -----------------------------------------------------------------------------------


W0729 09:47:09.403327  6944 deprecation.py:323] From <ipython-input-6-948e11d448d3>:10: max_pooling3d (from tensorflow.python.layers.pooling) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.MaxPooling3D instead.
W0729 09:47:09.550773  6944 deprecation.py:323] From <ipython-input-6-948e11d448d3>:21: batch_normalization (from tensorflow.python.layers.normalization) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
W0729 09:47:09.623094  6944 deprecation.py:323] From <ipython-input-6-948e11d448d3>:25: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W0729 09:47:09.845872  6944 deprecation.py:323] From <ipyt

{'learning_rate': 0.0001, 'keep_rate': 0.2, 'epochs': 201, 'batch_size': 128, 'kernel_size_1': 3, 'kernel_size_2': 3, 'pool_size': 2, 'dense_activation1': 'relu', 'dense_activation2': 'sigmoid'}
0 % complete
9 % complete
19 % complete
29 % complete
39 % complete
49 % complete
59 % complete
69 % complete
79 % complete
89 % complete
99 % complete
Testing Set Accuracy: 0.47999998927116394
Time elapsed:  0:01:02.799469
Index - 0
Value - 0.47999998927116394
Hyper-parameters {'learning_rate': 0.0001, 'keep_rate': 0.2, 'epochs': 201, 'batch_size': 128, 'kernel_size_1': 3, 'kernel_size_2': 3, 'pool_size': 2, 'dense_activation1': 'relu', 'dense_activation2': 'sigmoid'}
Total Run Time:  0:01:05.691883
